In [1]:
from FaceLoader import FaceLoader
from SocialProcessor import SocialProcessor

import vk_api
import mysql.connector
import json


%matplotlib inline

In [2]:
conn = mysql.connector.connect(user='face', host="localhost", password="Hr3mI3PM5JlFWrub", database='global_scrape', port=8889, collation="utf8mb4_general_ci")

In [3]:
credentials = {}
with open('credentials.json', 'r') as file:
    credentials = json.load(file)

In [4]:
def aunt_vk():
    return input(), True


def capt_vk(captcha):
    print(captcha.get_url())
    inp = input()
    captcha.try_again(inp)

In [5]:
vk_session = vk_api.VkApi(credentials['login'], credentials['password'], auth_handler=aunt_vk, captcha_handler=capt_vk)
vk_session.auth()

api = vk_session.get_api()

In [6]:
processor = SocialProcessor(conn, None)

In [7]:
already = set(processor.alreadyParsed()).union(set([i[0] for i in processor.getAllTasks()]))

In [8]:
len(already)

236

In [9]:
groups = [
    "phystech_tur",
    "love_mipt",
    "mipt_overhear",
    "miptru",
    "profkom_phystech"
]

In [10]:
for_process = set()
for group in groups:
    offset = 0
    count = 0
    initial = api.groups.getMembers(group_id=group)
    count = int(initial['count'])
    while count>0:
        for_process = for_process.union(set(list(map(str,initial['items']))))
        count-=len(initial['items'])
        offset+=len(initial['items'])
        initial = api.groups.getMembers(group_id=group, offset=offset)
for_process = for_process - already

In [11]:
len(for_process)

39491

In [12]:
for i in for_process:
    processor.addTask(i, "vk")